In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import datetime
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import optuna
import eli5
from eli5.sklearn import PermutationImportance
import pickle
import warnings
warnings.simplefilter("ignore")

dataset - данные с гитхаб (сбор с банки.ру)

dataset_3 - мэтчинг номера лицензии банка с его названием

In [2]:
df = pd.read_csv('dataset.csv')

In [3]:
df2 = pd.read_excel('dataset_3.xlsx', engine='openpyxl')

In [4]:
df2 = df2[['Lic', 'fact', 'date']]
df2['date'] = df2['date'].dt.to_period('M')
df = df.sort_values(by=['дата', 'лицензия'])
df['дата'] = pd.to_datetime(df['дата']).dt.to_period('M')

for i in tqdm(range(len(df))):
    tmp1 = df.iloc[i, 0]
    tmp2 = df.iloc[i, 1]
    try:
        a = df2.loc[(df2['Lic']==tmp1)&(df2['date']==tmp2+3), 'fact'].values[0]
        df.iloc[i, -1] = a
    except:
        try:
            a = df2.loc[(df2['Lic']==tmp1)&(df2['date']==tmp2+2), 'fact'].values[0]
            df.iloc[i, -1] = a
        except:
            try:
                a = df2.loc[(df2['Lic']==tmp1)&(df2['date']==tmp2+1), 'fact'].values[0]
                df.iloc[i, -1] = a
            except:
                df.iloc[i, -1] = 0

df = df[df['лицензия'].isna()==False]
df.drop_duplicates(subset=['лицензия', 'дата'], inplace=True)

100%|██████████| 72439/72439 [01:50<00:00, 653.04it/s]


In [5]:
df['default'].value_counts()

default
0.0       69285
отозв.      807
ликв.       127
Name: count, dtype: int64

In [6]:
df.to_csv('dataset_1.csv')